To run this notebook you first need to run the notebook `experiments/run_experiments.ipynb` or obtain the data generated for such notebook in the corresponding folders
---

In [1]:
from json import dump, load
import networkx as nx

## Load gene graphs

f = open(f'../gene_graphs/vertices_inv.json', 'r')
vertices_inv = load(f)
f.close()
vertices_inv = {int(k): tuple(v) for k,v in vertices_inv.items()}

components = list()
for i in range(19641): ## Number of gene_graphs
    components.append(dict())
    components[i]['graph'] = nx.read_edgelist(f'../gene_graphs/graphs/component_{i+1}.edgelist', delimiter=':', create_using=nx.DiGraph, nodetype=int)
    components[i]['len'] = len(components[i]['graph'])
    
    f = open(f'../gene_graphs/sources/component_{i+1}.json', 'r')
    components[i]['sources'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/targets/component_{i+1}.json', 'r')
    components[i]['targets'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/vertex_constrains/component_{i+1}.json', 'r')
    components[i]['vertex_constrains'] = set(load(f))
    f.close()
    
    f = open(f'../gene_graphs/transcript_paths/component_{i+1}.json', 'r')
    components[i]['transcript_paths'] = load(f)
    f.close()
    

In [2]:
## Load experiments from json files
for i, component in enumerate(components):
    if component['len'] > 2:
        file = open(f'../safe_paths_json/component_{i+1}.json', "r")
        dd = load(file)
        component.update(dd)
        file.close()

In [3]:
## If the first or last vertex are internals of a unitig, extend them by one vertex, respectively
def extend_unitig(G, in_neighbor, unitig):
    first = unitig[0]
    last = unitig[-1]
    
    if G.in_degree(first) == 1:
        unitig = [in_neighbor[first]] + unitig
    if G.out_degree(last) == 1:
        unitig = unitig + list(G.neighbors(last))
    
    return unitig
    
## Given a DAG G, it computes maximal unitigs of G 
## that are safe paths of every ST-path cover
def compute_maximal_safe_unitigs(G, S , T): 
    visited = dict(zip(G.nodes, len(G)*[False]))
    normal_unitigs = list()
    
    for v in nx.topological_sort(G):
        if not visited[v]:
            visited[v] = True
            
            if G.out_degree(v) == 1 and G.in_degree(v) <= 1:
                normal_unitig = [v] # We start a unitig
                
                t = list(G.neighbors(v))[0]
                while G.in_degree(t) == 1 and G.out_degree(t) == 1: ## While internal vertex have in/outdegree=1, we append those vertices to the unitig
                    visited[t] = True
                    normal_unitig.append(t)
                    t = list(G.neighbors(t))[0]
                 
                if G.in_degree(v) == 1 and G.out_degree(v) == 0:
                    visited[t] = True
                    normal_unitig.append(t)
                
                normal_unitigs.append(normal_unitig)
                
            elif G.out_degree(v) == 0:
                normal_unitigs.append([v])
      
    in_neighbor = dict()
    for u in G:
        for v in list(G.neighbors(u)):
            if G.in_degree(v) == 1:
                in_neighbor[v] = u
            
        
    normal_unitigs = list(
        map(
            lambda unitig: extend_unitig(G, in_neighbor, unitig),
            normal_unitigs
        )
    )
    
    
    normal_unitigs = list(filter(lambda unitig: len(unitig)>2, normal_unitigs))
    
    ## Now we cut them so, these unitigs are ST safe
    ST_unitigs = list()
    st = False
    for normal_unitig in normal_unitigs:
        last_s = 0
        s = 0
        for i, v in enumerate(normal_unitig):
            
            if v in S:
                last_s = i
                if st:
                    st = False
                    s = i
                
            if i == len(normal_unitig)-1:
                ST_unitigs.append(normal_unitig[s: i+1])
                s = last_s
                
            elif v in T:
                if last_s != s:
                    ST_unitigs.append(normal_unitig[s: i+1])
                    s = last_s
                elif i+1 < len(normal_unitig) and normal_unitig[i+1] in S:
                    ST_unitigs.append(normal_unitig[s: i+1])
                    s = i+1
                    
            st = v in T and v in S
    
    ST_unitigs =  list(filter(lambda unitig: len(unitig)>1, ST_unitigs))
    return ST_unitigs

In [4]:
def interval_length(interval):
    return interval[1]-interval[0]+1

## It computes the base length of a transcript
def base_length(contig, vertices_inv):
    length = 0
    for v in contig:
        length += interval_length(vertices_inv[v])
    return length

In [5]:
## Given a list of n base contigs and n improved contigs, this function returns
## a list of l elements, where the i-th element is the difference of length between
## the i-th base_contig and the longest improved_contig to which the base_contig
## aligns completely, or 0 if not such improved contig exists
## Two lists are returned, the first in vertex space and the second in base space
def relative_improvement(base_contigs, improved_contigs, vertices_inv):
    max_vertex_lengths = list()
    max_base_lengths = list()
    
    base_contigs_staring_at = dict()
    ## For every start vertex in a base_contig, put the corresponding base_contig in the dict
    for c, base_contig in enumerate(base_contigs):
        v = base_contig[0]
        if base_contigs_staring_at.get(v, None) is None:
            base_contigs_staring_at[v] = list()
        base_contigs_staring_at[v].append(c)
        
        
    max_vertex_lengths = [0]*len(base_contigs)
    max_base_lengths = [0]*len(base_contigs)
    
    for improved_contig in improved_contigs:
        for i,v in enumerate(improved_contig):
            if base_contigs_staring_at.get(v, None) is not None:
                for c in base_contigs_staring_at[v]:
                    base_contig = base_contigs[c]
                    if i+len(base_contig) <= len(improved_contig):
                        aligns = True
                        for j in range(len(base_contig)):
                            if base_contig[j] != improved_contig[j+i]:
                                aligns = False
                                break
                        if aligns:
                            max_vertex_lengths[c] = max(max_vertex_lengths[c], len(improved_contig))
                            max_base_lengths[c] = max(max_base_lengths[c], base_length(improved_contig, vertices_inv))
                            
        
    
    return max_vertex_lengths, max_base_lengths

In [6]:
for i, component in enumerate(components):
    component['unitigs'] = compute_maximal_safe_unitigs(component['graph'], component['sources'], component['targets'])

In [7]:
## Filter unitigs included inside other unitigs, needs relative_improvement function
for i, component in enumerate(components):
    if component['len'] > 2:
        
        large, _ = relative_improvement(component['unitigs'], component['unitigs'] , vertices_inv)
        lengths = list(map(lambda unitig: len(unitig), component['unitigs']))
        
        filter_unitigs = list()
        for j in range(len(large)):
            if large[j] <= lengths[j]:
                filter_unitigs.append(component['unitigs'][j])
        component['unitigs'] = filter_unitigs

In [8]:
## The e_size is computed for every transcript path, and for every base in this transcript path
## For every transcript EVERY contig intersecting with that contig is considered
def compute_e_size(transcript_paths, contigs, vertices_inv):
    contigs_through = dict()
    ## For every vertex in a contig, compute the contigs using that vertex and its index in the corresponding contig
    for c, contig in enumerate(contigs):
        for i, v in enumerate(contig):
            if contigs_through.get(v, None) is None:
                contigs_through[v] = list()
            contigs_through[v].append((c,i))
    
    e_size_per_transcript_path = list()
    e_size_per_transcript_path_vertex = list()
    for transcript_path in transcript_paths:
        
        length = 0
        e_sum = 0
        e_sum_vertex = 0
        for j, v in enumerate(transcript_path):
            exon_length = interval_length(vertices_inv[v])
            length += exon_length
            
            if contigs_through.get(v, None) is not None:
                total_length_intersections = 0
                total_length_intersections_vertex = 0
                
                for c,i in contigs_through[v]:
                    
                    contig = contigs[c]
                    l_p = i
                    while l_p >= 0 and (j-(i-l_p)) >= 0 and contig[l_p] == transcript_path[(j-(i-l_p))]:
                        l_p -= 1
                    
                    l_p += 1
                    
                    r_p = i
                    while r_p < len(contig) and (j-(i-r_p)) < len(transcript_path) and contig[r_p] == transcript_path[(j-(i-r_p))]:
                        r_p += 1
                    r_p -= 1
                    
                    intersection = contig[l_p:r_p+1]
                    total_length_intersections += base_length(intersection, vertices_inv)
                    total_length_intersections_vertex += len(intersection)
                    
                    
                e_sum += exon_length*(total_length_intersections/len(contigs_through[v]))
                e_sum_vertex += total_length_intersections_vertex/len(contigs_through[v])
                    
                
        e_size_per_transcript_path.append(e_sum/length)
        e_size_per_transcript_path_vertex.append(e_sum_vertex/len(transcript_path))
    
    return e_size_per_transcript_path_vertex, e_size_per_transcript_path

In [9]:
## Returns a list t_p of size len(contigs) such that 
## tp[i] is true iff contigs[i] aligns completely to some
## transcript path
def true_positives(transcript_paths, contigs, vertices_inv):
    t_p = [False]*len(contigs)
    
    contigs_staring_at = dict()
    ## For every start vertex in a contig, put the corresponding contig in the dict
    for c, contig in enumerate(contigs):
        v = contig[0]
        if contigs_staring_at.get(v, None) is None:
            contigs_staring_at[v] = list()
        contigs_staring_at[v].append(c)
    
    
    for transcript_path in transcript_paths:
        
        for i,v in enumerate(transcript_path):
            if contigs_staring_at.get(v, None) is not None:
                not_aligning_contigs = list()
                
                for c in contigs_staring_at[v]: ## All contigs evaluated start at some vertex of the transcript
                    contig = contigs[c]
                    if i+len(contig) <= len(transcript_path): ## They also end at some vertex of the contig
                        aligns = True
                        for j in range(len(contig)):
                            if contig[j] != transcript_path[j+i]:
                                aligns = False
                                break
                        if aligns:
                            t_p[c] = True
                        else:
                            not_aligning_contigs.append(c)
                    else:
                        not_aligning_contigs.append(c)
                
                contigs_staring_at[v] = not_aligning_contigs
    
    
    return t_p

In [10]:
## Returns a list max_cov of length len(transcript_paths)
## such that max_cov[i] is the maximum of (vertices, bases)
## covered by any contig in contigs INTERSECTING to transcript_path[i]
def max_covered_by_a_contig(transcript_paths, contigs, vertices_inv):
    max_cov_vertex = list()
    max_cov_bases = list()
    
    contigs_through = dict()
    
    ## For every vertex in a contig, compute the contigs using that vertex and its index in the corresponding contig
    for c, contig in enumerate(contigs):
        for i, v in enumerate(contig):
            if contigs_through.get(v, None) is None:
                contigs_through[v] = list()
            contigs_through[v].append((c,i))
            
    
    for transcript_path in transcript_paths:
        max_bases = 0
        max_vertex = 0
        
        for j,v in enumerate(transcript_path):
            
            
            
            if contigs_through.get(v, None) is not None:
                
                for c,i in contigs_through[v]:
                    
                    contig = contigs[c]
                    l_p = i
                    while l_p >= 0 and (j-(i-l_p)) >= 0 and contig[l_p] == transcript_path[(j-(i-l_p))]:
                        l_p -= 1
                    
                    l_p += 1
                    
                    r_p = i
                    while r_p < len(contig) and (j-(i-r_p)) < len(transcript_path) and contig[r_p] == transcript_path[(j-(i-r_p))]:
                        r_p += 1
                    r_p -= 1
                    
                    
                    intersection = contig[l_p:r_p+1]
                    length_intersection_vertex = len(intersection)
                    length_intersection_bases = base_length(intersection, vertices_inv)
                    
                    
                    
                    max_bases = max(max_bases, length_intersection_bases)
                    max_vertex = max(max_vertex, length_intersection_vertex)
                    
                    
                    
        max_cov_bases.append(max_bases)
        max_cov_vertex.append(max_vertex)
    
    return max_cov_vertex, max_cov_bases

In [11]:
## Compute statistics for unitigs
for i, component in enumerate(components):
    if component['len'] > 2:
        transcript_paths = component['transcript_paths']
        unitigs = component['unitigs']
            
        e_sizes_vertex, e_size_bases = compute_e_size(transcript_paths, unitigs, vertices_inv)
        
        unitigs_lengths = list(map(lambda unitigs: float(base_length(unitigs, vertices_inv)) , unitigs))
            
        max_cov_vertex, max_cov_bases = max_covered_by_a_contig(transcript_paths, unitigs, vertices_inv)
            
        tps = true_positives(transcript_paths,  unitigs, vertices_inv)
        true_pos = list()
        false_pos = list()
        for i, tp in enumerate(tps):
            unitig = unitigs[i]
            if tp:
                true_pos.append(unitig)
            else:
                false_pos.append(unitig)

        component['e_sizes_vertex'] = e_sizes_vertex
        component['e_size_bases'] = e_size_bases
        
        
        component['unitigs_lengths'] = unitigs_lengths
        
        component['max_cov_vertex'] = max_cov_vertex
        component['max_cov_bases'] = max_cov_bases
        
        component['true_positives'] = true_pos
        component['false_positives'] = false_pos

In [12]:
def to_float(l):
    return list(map(lambda e: float(e), l))


## Compute evaluation metrics by gene graph
for i, component in enumerate(components):
    if component['len'] > 2:
        n = len(component['graph'])
        uni = component['unitigs']
        transcript_paths = component['transcript_paths']
        experiments = component['experiments']
        for l in experiments:
            experiment = experiments[l]
            
            safe_paths = experiment['safe_paths']
            
            e_sizes_vertex, e_size_bases = compute_e_size(transcript_paths, safe_paths, vertices_inv)
            
            safe_paths_lengths = list(map(lambda contig: float(base_length(contig, vertices_inv)) , safe_paths))
            
            max_cov_vertex, max_cov_bases = max_covered_by_a_contig(transcript_paths, safe_paths, vertices_inv)
            
            improvement_vertex, improvement_base = relative_improvement(uni, safe_paths, vertices_inv)
            
            tps = true_positives(transcript_paths,  safe_paths, vertices_inv)
            true_pos = list()
            false_pos = list()
            for i, tp in enumerate(tps):
                safe_path = safe_paths[i]
                if tp:
                    true_pos.append(safe_path)
                else:
                    false_pos.append(safe_path)
            
            experiment['e_sizes_vertex'] = to_float(e_sizes_vertex)
            experiment['e_size_bases'] = to_float(e_size_bases)
            
            experiment['safe_paths_lengths'] = safe_paths_lengths
            
            experiment['max_cov_vertex'] = to_float(max_cov_vertex)
            experiment['max_cov_bases'] = to_float(max_cov_bases)
            
            experiment['true_positives'] = true_pos
            experiment['false_positives'] = false_pos
            
            experiment['impr_vertex'] = improvement_vertex
            experiment['impr_base'] = improvement_base

In [13]:
from json import dump
## Store these results to a file in json format

for i, component in enumerate(components):
    if component['len'] > 2:
        d = dict()
        d['width'] = component['width']
        d['number_of_transcripts'] = len(component['transcript_paths'])
        d['experiments'] = component['experiments']
        d['experiments_two_finger'] = component['experiments_two_finger']
        d['experiments_unoptimized'] = component['experiments_unoptimized']
        d['experiments_heuristic'] = component['experiments_heuristic']
        
        d['unitigs'] = component['unitigs']
        d['e_sizes_vertex'] = component['e_sizes_vertex']
        d['e_size_bases'] = component['e_size_bases']
        d['unitigs_lengths'] = component['unitigs_lengths']
        d['max_cov_vertex'] = component['max_cov_vertex']
        d['max_cov_bases'] = component['max_cov_bases']
        d['true_positives'] = component['true_positives']
        d['false_positives'] = component['false_positives']
        
        file = open(f'../safe_paths_json/component_{i+1}.json' , 'w')
        dump(d, file)
        file.close()